In [ ]:
%matplotlib inline
import warnings
from datetime import date
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
sns.set(style='darkgrid', context='notebook', color_codes=True)

In [ ]:
import os,sys
module_path = os.path.abspath(os.path.join("/home/miki/Desktop/Deployment/time-series/src"))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from feat import *

In [ ]:
DATAPATH = "/home/miki/Desktop/Deployment/time-series/data/raw/time.csv"
df = data_load(DATAPATH)

In [ ]:
df

In [ ]:
summary= summary_statistics(df['case'])
sums = summary_statistics(df['death'])

In [ ]:
summary
## slight deviation of the data from the mean indicated and hence not normally distributed 
## median less than the mean indicates the skeweness of the data toward right
## the minimum of the case is 0 and the maximum number of ifection reported is 210
## the difference between the 75% IQR and the maximum case indicates that there are a potential 
## outliers in data

In [ ]:
sums

In [ ]:
df = check_index(df)

In [ ]:
df

In [ ]:
df.set_index('date',drop=True, inplace=True)
df.index.freq = 'M'

In [ ]:
df.plot(subplots=True, figsize=(14,8));

In [ ]:
def plot_correlogram(x, lags=None, title=None):    
    lags = min(10, int(len(x)/5)) if lags is None else lags
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(14, 8))
    x.plot(ax=axes[0][0])
    q_p = np.max(q_stat(acf(x, nlags=lags), len(x))[1])
    stats = f'Q-Stat: {np.max(q_p):>8.2f}\nADF: {adfuller(x)[1]:>11.2f}'
    axes[0][0].text(x=.02, y=.85, s=stats, transform=axes[0][0].transAxes)
    probplot(x, plot=axes[0][1])
    mean, var, skew, kurtosis = moment(x, moment=[1, 2, 3, 4])
    s = f'Mean: {mean:>12.2f}\nSD: {np.sqrt(var):>16.2f}\nSkew: {skew:12.2f}\nKurtosis:{kurtosis:9.2f}'
    axes[0][1].text(x=.02, y=.75, s=s, transform=axes[0][1].transAxes)
    plot_acf(x=x, lags=lags, zero=False, ax=axes[1][0])
    plot_pacf(x, lags=lags, zero=False, ax=axes[1][1])
    axes[1][0].set_xlabel('Lag')
    axes[1][1].set_xlabel('Lag')
    fig.suptitle(title, fontsize=20)
    fig.tight_layout()
    fig.subplots_adjust(top=.9)

In [ ]:
plot_correlogram(df['case'], lags=12)

In [ ]:
plot_correlogram(df['death'], lags=12)

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adfuller_test(series, sig=0.05, name=''):
    res = adfuller(series, autolag='AIC')    
    p_value = round(res[1], 3) 

    if p_value <= sig:
        print(f" {name} : P-Value = {p_value} => Stationary. ")
    else:
        print(f" {name} : P-Value = {p_value} => Non-stationary.")

for name, column in df.iteritems():
    adfuller_test(column, name=column.name)

In [ ]:
train = df.iloc[:48]

In [ ]:
train

In [ ]:
model = VARMAX(df.iloc[:48], order=(1,1), trend='c').fit(maxiter=1000)

In [ ]:
print(model.summary())

In [ ]:
model.plot_diagnostics(variable=0, figsize=(14,8), lags=24)
plt.gcf().suptitle('Industrial Production - Diagnostics', fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=.9);

In [ ]:
model.plot_diagnostics(variable=1, figsize=(14,8), lags=24)
plt.title('Sentiment - Diagnostics');

In [ ]:
median_change = df.diff().quantile(.5).tolist()
model.impulse_responses(steps=12, impulse=median_change).plot.bar(subplots=True);

In [ ]:
import matplotlib.transforms as mtransforms

start = 43
preds = model.predict(start=48, end=len(df)-1)
preds.index = df.index[48:]

fig, axes = plt.subplots(nrows=2, figsize=(12, 8))

df.case.iloc[start:].plot(ax=axes[0], label='actual', title='Industrial Production')
preds.case.plot(label='predicted', ax=axes[0])
trans = mtransforms.blended_transform_factory(axes[0].transData, axes[0].transAxes)
axes[0].legend()
axes[0].fill_between(x=df.index[48:], y1=0, y2=1, transform=trans, color='grey', alpha=.5)

trans = mtransforms.blended_transform_factory(axes[0].transData, axes[1].transAxes)
df.death.iloc[start:].plot(ax=axes[1], label='actual', title='Sentiment')
preds.death.plot(label='predicted', ax=axes[1])
axes[1].fill_between(x=df.index[48:], y1=0, y2=1, transform=trans, color='grey', alpha=.5)
fig.tight_layout();

In [ ]:
forecast = model.forecast(steps=24)

fig, axes = plt.subplots(nrows=2, figsize=(12, 8))

df.case.plot(ax=axes[0], label='actual', title='Liquor')
preds.case.plot(label='predicted', ax=axes[0])
axes[0].legend()

df.death.plot(ax=axes[1], label='actual', title='Sentiment')
preds.death.plot(label='predicted', ax=axes[1])
axes[1]
fig.tight_layout();